# Support Vector Machines 
### with Hyperparameter Tuning

### Hinweis:
Anders als in den Praktika und Vorlesungen verwenden wir hier nicht `Scikit-learn`, sondern die Bibliothek `cuML` (CUDA Machine Learning). Diese ermöglicht es SVMs auf der GPU zu trainieren, was den Trainingsprozess extrem beschleunigt. An der Herangehensweise und der Art, wie wir die Hyperparameter tunen, ändert sich dadurch nichts.

In [1]:
import cudf
import cupy as cp
import numpy as np
import time
from cuml.svm import SVC
from cuml.metrics import accuracy_score
from sklearn.model_selection import ParameterSampler, ParameterGrid
from cuml.model_selection import GridSearchCV
import joblib

DATAPATH = '../Data/'
MODELPATH = '../Data/Models/SVM/'
MODELPATHSKLEARN = '../Data/Models/SVM/SVM_sklearn/'

train_set = cudf.read_csv(DATAPATH + 'train_set.csv')
train_labels = cudf.read_csv(DATAPATH + 'train_labels.csv')
test_set = cudf.read_csv(DATAPATH + 'test_set.csv')
test_labels = cudf.read_csv(DATAPATH + 'test_labels.csv')

train_labels = train_labels.squeeze()
test_labels = test_labels.squeeze()


# WITHOUT Hyperparameter Tuning

In [18]:
clf = SVC()

clf.fit(train_set, train_labels)

test_pred = clf.predict(test_set)
accuracy_score = accuracy_score(test_labels, test_pred)

joblib.dump(clf, MODELPATH + 'SVM_no_hyper.pkl')

print(f'Accuracy: {accuracy_score}')

Accuracy: 0.7532467246055603


# With (Random) Hyperparameter Tuning

In [2]:
param_distributions = [
    {
        'kernel': ['linear'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['poly'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'degree': [2, 3, 4, 5, 6], # Macht natrülich nur Sinn, wenn degree > 1 da sonst linear
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['rbf'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['sigmoid'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'coef0': np.linspace(-1, 1, 10),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    }
]

In [3]:
n_iter = 100

best_accuracy = 0
best_params = None

start_time = time.time()

all_sampled_params = []
for params in param_distributions:
    sampler = ParameterSampler(params, n_iter=n_iter, random_state=42)
    sampled_params = list(sampler)
    all_sampled_params.extend(sampled_params)

clf = SVC()

counter = 0

for params in all_sampled_params:
    try:
        clf.set_params(**params)
        clf.fit(train_set, train_labels)
        predictions = clf.predict(test_set)
        accuracy = accuracy_score(test_labels, predictions)
        counter += 1

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params
    except ValueError as e:
        print(f"Skipping invalid parameter combination: {params}")
        print(f"Error: {e}")


end_time = time.time()

best_model = SVC(**best_params)
best_model.fit(train_set, train_labels) 
test_predictions = best_model.predict(test_set)
test_accuracy = accuracy_score(test_labels, test_predictions)

joblib.dump(best_model, MODELPATH + 'SVM_para_sampl.pkl')

['../Data/Models/SVM/SVM_para_sampl.pkl']

In [4]:
print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")
print(f"Time taken: {end_time - start_time:.2f} seconds")
print(f"Test accuracy with best model: {test_accuracy}")

Best parameters: {'tol': np.float64(0.1), 'max_iter': 4000, 'kernel': 'poly', 'gamma': np.float64(0.1), 'degree': 5, 'class_weight': 'balanced', 'C': np.float64(1.0)}
Best accuracy: 0.7662337422370911
Time taken: 39.86 seconds
Test accuracy with best model: 0.701298713684082


# Further Hyperparameter Tuning with Parameter Grid

In [7]:
param_grid = {
        'kernel': ['poly'],
        'C': [0.01, 0.1, 0.8, 0.9, 1, 1.1, 1.2, 2, 5, 10], 
        'tol': [0.01, 0.01, 0.1],
        'gamma': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5],
        'degree': [4, 5, 6],
        'max_iter': np.linspace(3900, 4100, 10),
        'class_weight': ['balanced'],
    },

best_accuracy = 0
best_params = None

start_time = time.time()

for params in ParameterGrid(param_grid):
    clf.set_params(**params)
    clf.fit(train_set, train_labels)
    predictions = clf.predict(test_set)
    accuracy = accuracy_score(test_labels, predictions)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params    

end_time = time.time()

best_model = SVC(**best_params)
best_model.fit(train_set, train_labels)
test_predictions = best_model.predict(test_set)

test_accuracy = accuracy_score(test_labels, test_predictions)

joblib.dump(best_model, MODELPATH + 'SVM_para_grid.pkl')

['../Data/Models/SVM/SVM_para_grid.pkl']

In [8]:
print("\n" + "="*50)
print("GRID SEARCH RESULTS")
print("="*50)
print("\nBest Parameters Found:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

print("\nModel Performance:")
print(f"  Validation Accuracy: {best_accuracy*100:.2f}%")

print(f"  Test Accuracy: {test_accuracy*100:.2f}%")
print(f"\nTraining Time: {end_time - start_time:.2f} seconds")
print("="*50)


GRID SEARCH RESULTS

Best Parameters Found:
  C: 1.2
  class_weight: balanced
  degree: 4
  gamma: 0.07
  kernel: poly
  max_iter: 3900.0
  tol: 0.1

Model Performance:
  Validation Accuracy: 78.57%
  Test Accuracy: 66.88%

Training Time: 442.50 seconds


# (Random) Hyperparameter Tuning with Cross-Validation

In [5]:
class CuMLRandomizedSearchCV:

    model = SVC()

    def __init__(self, param_distributions, n_iter=10, cv_method='kfold', k=5, random_state=42):
        self.param_distributions = param_distributions
        self.n_iter = n_iter
        self.cv_method = cv_method
        self.k = k
        self.random_state = random_state
        self.results = []
        self.best_params = None
        self.best_score = None

    def _sample_parameters(self):
        """Sample parameter combinations from the given distributions."""
        for param_distribution in self.param_distributions:
            sampler = ParameterSampler(param_distribution, n_iter=self.n_iter, random_state=self.random_state)
            for sampled_params in sampler:
                yield sampled_params

    def _train_and_evaluate(self, X, y, sampled_params):
        """Train and evaluate a model with given parameters using the chosen CV method."""
        scores = []
        
        if self.cv_method == 'kfold':
            scores = self._kfold_cv(X, y, sampled_params)
        elif self.cv_method == 'stratified':
            scores = self._stratified_kfold_cv(X, y, sampled_params)
        elif self.cv_method == 'loocv':
            scores = self._loocv(X, y, sampled_params)
        else:
            raise ValueError("Invalid cv_method. Choose 'kfold', 'stratified', or 'loocv'.")
        
        return np.mean(scores)

    def _kfold_cv(self, X, y, sampled_params):
        """Perform k-fold cross-validation."""
        from sklearn.model_selection import KFold

        kf = KFold(n_splits=self.k, shuffle=True, random_state=self.random_state)
        scores = []
        for fold, (train_index, val_index) in enumerate(kf.split(X)):
            print(f"  Training and evaluating fold {fold+1}/{self.k}...")
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            
            CuMLRandomizedSearchCV.model.set_params(**sampled_params)
            CuMLRandomizedSearchCV.model.fit(X_train, y_train)
            score = CuMLRandomizedSearchCV.model.score(X_val, y_val)
            scores.append(score)
        return scores

    def _stratified_kfold_cv(self, X, y, sampled_params):
        """Perform stratified k-fold cross-validation."""
        from cuml.model_selection import StratifiedKFold

        skf = StratifiedKFold(n_splits=self.k, random_state=self.random_state, shuffle=True)
        scores = []
        for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
            print(f"  Training and evaluating fold {fold+1}/{self.k}...")
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            
            CuMLRandomizedSearchCV.model.set_params(**sampled_params)
            CuMLRandomizedSearchCV.model.fit(X_train, y_train)
            score = CuMLRandomizedSearchCV.model.score(X_val, y_val)
            scores.append(score)
        return scores

    def _loocv(self, X, y, sampled_params):
        """Perform Leave-One-Out cross-validation."""
        from sklearn.model_selection import LeaveOneOut
        
        loo = LeaveOneOut()
        scores = []
        for i, (train_index, val_index) in enumerate(loo.split(X)):
            print(f"  Training and evaluating sample {i+1}/{len(X)}...")
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            
            CuMLRandomizedSearchCV.model.set_params(**sampled_params)
            CuMLRandomizedSearchCV.model.fit(X_train, y_train)
            score = CuMLRandomizedSearchCV.model.score(X_val, y_val)
            scores.append(score)
        return scores

    def fit(self, X, y):
        """Fit the randomized search CV to the data."""
        print("Starting Randomized Search CV...")
        for i, sampled_params in enumerate(self._sample_parameters()):
            print(f"Evaluating parameter set {i+1}/{self.n_iter * len(self.param_distributions)}: {sampled_params}")
            avg_score = self._train_and_evaluate(X, y, sampled_params)
            print(f"  Average CV score: {avg_score:.4f}")
            self.results.append({'params': sampled_params, 'score': avg_score})

        best_result = max(self.results, key=lambda x: x['score'])
        self.best_params = best_result['params']
        self.best_score = best_result['score']
        self.X_train = X
        self.y_train = y
        print("Randomized Search CV complete.")
        print(f"Best parameters found: {self.best_params}")
        print(f"Best CV score: {self.best_score:.4f}")
        return self

    def predict(self, train_set, train_labels, test_set):
        """Predict using the best found parameters."""
        print("Training final model with best parameters...")
        final_model = SVC(**self.best_params)
        final_model.fit(train_set, train_labels)
        print("Final model training complete.")
        return final_model.predict(test_set)

    def score(self, X, y):
        """Score the model's performance."""
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)
    
    def get_best_params(self):
        """Return the best parameters found by the randomized search."""
        return self.best_params
    
    def get_best_score(self):
        """Return the best score found by the randomized search."""
        return self.best_score
    
    def get_results(self):
        """Return the results of the randomized search."""
        return self.results
    
    def get_best_model(self):
        """Return the best model found by the randomized search."""
        return SVC(**self.best_params)
    
    def save_best_model(self, train_set, train_labels):
        """Save the best model to a file."""
        final_model = SVC(**self.best_params)
        final_model.fit(train_set, train_labels)
        joblib.dump(final_model, DATAPATH + 'SVM_rnd_search_CV.pkl')

In [14]:
searcher = CuMLRandomizedSearchCV(param_distributions=param_distributions, n_iter=20, cv_method='loocv', k=5, random_state=60)
searcher.fit(train_set, train_labels)

Starting Randomized Search CV...
Evaluating parameter set 1/80: {'tol': np.float64(1e-07), 'max_iter': 5000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(1.0)}
  Training and evaluating sample 1/800...
  Training and evaluating sample 2/800...
  Training and evaluating sample 3/800...
  Training and evaluating sample 4/800...
  Training and evaluating sample 5/800...
  Training and evaluating sample 6/800...
  Training and evaluating sample 7/800...
  Training and evaluating sample 8/800...
  Training and evaluating sample 9/800...
  Training and evaluating sample 10/800...
  Training and evaluating sample 11/800...
  Training and evaluating sample 12/800...
  Training and evaluating sample 13/800...
  Training and evaluating sample 14/800...
  Training and evaluating sample 15/800...
  Training and evaluating sample 16/800...
  Training and evaluating sample 17/800...
  Training and evaluating sample 18/800...
  Training and evaluating sample 19/800...
  Training an

RuntimeError: The work in this thread was cancelled.
Obtained 33 stack frames
#1 in /home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/pylibraft/common/handle.cpython-312-x86_64-linux-gnu.so(+0x3a82b) [0x7f6136a9182b]
#2 in /home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/pylibraft/common/handle.cpython-312-x86_64-linux-gnu.so(+0x41700) [0x7f6136a98700]
#3 in /home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/cuml/svm/svc.cpython-312-x86_64-linux-gnu.so(+0x4707e) [0x7f60eafa307e]
#4 in /home/chris/miniconda3/envs/rapids-24.12/bin/python(+0x113339) [0x5639a8237339]
#5 in /home/chris/miniconda3/envs/rapids-24.12/bin/python: PyEval_EvalCode +0xa1 [0x5639a83de741]
#6 in /home/chris/miniconda3/envs/rapids-24.12/bin/python(+0x2d5ece) [0x5639a83f9ece]
#7 in /home/chris/miniconda3/envs/rapids-24.12/bin/python(+0x112f8e) [0x5639a8236f8e]
#8 in /home/chris/miniconda3/envs/rapids-24.12/bin/python(+0x2d099f) [0x5639a83f499f]
#9 in /home/ch

In [13]:
best_model = SVC(**searcher.get_best_params())
best_model.fit(train_set, train_labels)
test_predictions = best_model.predict(test_set)
test_accuracy = accuracy_score(test_labels, test_predictions)

print("\n" + "="*50)
print("RANDOMIZED SEARCH CV RESULTS")
print("="*50)
print("\nBest Parameters Found:")
for param, value in searcher.get_best_params().items():
    print(f"  {param}: {value}")

print("\nModel Performance:")
print(f"  Validation Accuracy: {searcher.get_best_score()*100:.2f}%")
print(f"  Test Accuracy: {test_accuracy*100:.2f}%")
print("="*50)


RANDOMIZED SEARCH CV RESULTS

Best Parameters Found:
  tol: 1e-07
  max_iter: 5000
  kernel: linear
  class_weight: balanced
  C: 1.0

Model Performance:
  Validation Accuracy: 83.00%
  Test Accuracy: 74.03%


# Further Tuning with GridSearchCV

In [15]:
param_gridCV = {
        'kernel': ['poly'],
        'C': [0.01, 0.1, 0.8, 0.9, 1, 1.1, 1.2, 2, 5, 10], 
        'tol': [0.01, 0.01, 0.1],
        'gamma': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5],
        'degree': [4, 5, 6],
        'max_iter': np.linspace(3900, 4100, 10),
        'class_weight': ['balanced'],
    }

In [18]:
from cuml.model_selection import GridSearchCV
svc_tuned = SVC()
grid_search = GridSearchCV(svc_tuned, param_gridCV, cv=5)
grid_search.fit(train_set, train_labels)


TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy().